<a href="https://colab.research.google.com/github/steelpipe75/learn-langgraph/blob/main/LangGraph_DFA/LangGraph_DFA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph体験 : LLMなし、LangGraphでDFAを作ってみる

https://qiita.com/steelpipe75/items/b5a49f5bb3203abf6fb0

**有限オートマトン（DFA）の設計**  
0 と 1 のみからなる文字列を受け取り、1 の出現回数が奇数回であるときに受理する

In [1]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 19.5 MB/s eta 0:00:00


In [2]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00


In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class State(TypedDict):
    input: str
    character: str | None

def next(state: State):
    if state["input"] == "":
        next_input = ""
        character = None
    else:
        next_input = state["input"][1:]
        character = state["input"][0]

    return {"input": state["input"][1:], "character" : character}

def accept(state: State):
    print("-- accept! --")
    return {"input": "", "character": None}

def reject(state: State):
    print("-- reject! --")
    return {"input": "", "character": None}

def routing_even(state: State):
    character = state["character"]
    if character == None:
        return "accept"
    elif character == "1":
        return "odd"
    elif character == "0":
        return "even"
    else:
        return "reject"

def routing_odd(state: State):
    character = state["character"]
    if character == None:
        return "reject"
    elif character == "1":
        return "even"
    elif character == "0":
        return "odd"
    else:
        return "reject"

graph_builder = StateGraph(State)

graph_builder.add_node("odd",    next)
graph_builder.add_node("even",   next)
graph_builder.add_node("accept", accept)
graph_builder.add_node("reject", reject)

graph_builder.set_entry_point("odd")
graph_builder.set_finish_point("accept")
graph_builder.set_finish_point("reject")

graph_builder.add_conditional_edges(
    source="even",
    path=routing_even,
    path_map={"accept": "accept", "reject": "reject", "even": "even", "odd": "odd"},
)
graph_builder.add_conditional_edges(
    source="odd",
    path=routing_odd,
    path_map={"reject": "reject", "even": "even", "odd": "odd"},
)

graph = graph_builder.compile()

In [4]:
# graph.get_graph().print_ascii() # エラーになる、、、一旦コメントにして無効化

In [5]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	odd(odd)
	even(even)
	accept(accept)
	reject(reject)
	__end__([<p>__end__</p>]):::last
	__start__ --> odd;
	even -.-> accept;
	even -.-> odd;
	even -.-> reject;
	odd -.-> even;
	odd -.-> reject;
	accept --> __end__;
	reject --> __end__;
	even -.-> even;
	odd -.-> odd;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [6]:
graph.invoke({"input": "11"}, debug=True) # 偶数 : 拒否

[-1:checkpoint] State at the end of step -1:
{}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'input': '11'}
[0:writes] Finished step 0 with writes to 1 channel:
- input -> '11'
[0:checkpoint] State at the end of step 0:
{'input': '11'}
[1:tasks] Starting 1 task for step 1:
- odd -> {'input': '11'}
[1:writes] Finished step 1 with writes to 2 channels:
- input -> '1'
- character -> '1'
[1:checkpoint] State at the end of step 1:
{'character': '1', 'input': '1'}
[2:tasks] Starting 1 task for step 2:
- even -> {'character': '1', 'input': '1'}
[2:writes] Finished step 2 with writes to 2 channels:
- input -> ''
- character -> '1'
[2:checkpoint] State at the end of step 2:
{'character': '1', 'input': ''}
[3:tasks] Starting 1 task for step 3:
- odd -> {'character': '1', 'input': ''}
[3:writes] Finished step 3 with writes to 2 channels:
- input -> ''
- character -> None
[3:checkpoint] State at the end of step 3:
{'character': None, 'input': ''}
[4:tasks] Starting 1 task for step 4:
- re

{'input': '', 'character': None}

In [7]:
graph.invoke({"input": "1"}) # 奇数 : 受理

-- accept! --


{'input': '', 'character': None}

In [8]:
graph.invoke({"input": "01010"}) # 偶数 : 拒否

-- reject! --


{'input': '', 'character': None}

In [9]:
graph.invoke({"input": "11111"}) # 奇数 : 受理

-- accept! --


{'input': '', 'character': None}

In [10]:
graph.invoke({"input": ""}) # 偶数 : 拒否

-- reject! --


{'input': '', 'character': None}

In [11]:
graph.invoke({"input": "1a"}) # 異常系 : 拒否

-- reject! --


{'input': '', 'character': None}